<h3><strong>Porównanie algorytmu: Minimax vs Minimax z Alpha-Beta pruning</strong></h3>

In [11]:
import chess
import chess.svg
from IPython.display import display, SVG
import time

# Tabela wartości figur
piece_values = {
    chess.PAWN: 100,
    chess.KNIGHT: 300,
    chess.BISHOP: 300,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 0
}

piece_position = {
    chess.PAWN: [
        0,  0,  0,  0,  0,  0,  0,  0,
        50, 50, 50, 50, 50, 50, 50, 50,
        10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 10, 10, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ],

    chess.KNIGHT: [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50,
    ],

    chess.BISHOP: [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20,
    ],

    chess.ROOK: [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        0, -50,  0,  5,  5,  0, -50, 0
    ],

    chess.QUEEN: [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
        -5,  0,  5,  5,  5,  5,  0, -5,
        0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ],
    
    chess.KING: [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
        20, 20,  0,  0,  0,  0, 20, 20,
        20, 30, 10,  0,  0, 10, 30, 20
    ]
}


In [ ]:
def evaluate_board(position):
    score = 0
    for square in chess.SQUARES:
        piece = position.piece_at(square)
        if piece:
            piece_type = piece.piece_type
            if piece.color == chess.WHITE:
                mirrored = chess.square_mirror(square)
                score += piece_values[piece_type]
                score += piece_position[piece_type][mirrored] 
            else:
                score -= piece_values[piece_type]
                score -= piece_position[piece_type][square]
    return score/100

In [13]:
def minimax(position, depth, maximizingPlayer):
    if depth == 0 or position.is_game_over():
        return evaluate_board(position), None
    
    best_move = None
    if maximizingPlayer:
        maxEval = float('-inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax(position, depth-1, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
        return maxEval, best_move
    else:
        minEval = float('inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax(position, depth-1, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
        return minEval, best_move

In [14]:
def minimax_alpha_beta(position, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or position.is_game_over():
        return evaluate_board(position), None
    
    best_move = None
    if maximizingPlayer:
        maxEval = float('-inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax_alpha_beta(position, depth-1, alpha, beta, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return maxEval, best_move
    else:
        minEval = float('inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax_alpha_beta(position, depth-1, alpha, beta, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return minEval, best_move

In [15]:
board = chess.Board()
white = True  # True - ruch białych, False - ruch czarnych

start = time.time()
move_count = 0

while not board.is_game_over():
    score, move = minimax_alpha_beta(board, 4, float('-inf'), float('inf'), white)
    
    if move is None:
        print("Brak dostępnych ruchów!")
        break
        
    board.push(move)

    white = not white
    move_count += 1

end = time.time()

print(f"Wynik: {board.result()}, liczba ruchów: {move_count}")
print(f"Czas wykonania dla Alpha–beta pruning: {end - start:.2f} sekund")

board.reset()

start = time.time()
move_count = 0

while not board.is_game_over():
    score, move = minimax(board, 4, white)
    
    if move is None:
        print("Brak dostępnych ruchów!")
        break
        
    board.push(move)

    white = not white
    move_count += 1

end = time.time()

print(f"Wynik: {board.result()}, liczba ruchów: {move_count}")
print(f"Czas wykonania dla Minimax bez optymalizacji: {end - start:.2f} sekund")


Wynik: 0-1, liczba ruchów: 48
Czas wykonania dla Alpha–beta pruning: 26.87 sekund
Wynik: 0-1, liczba ruchów: 48
Czas wykonania dla Minimax bez optymalizacji: 584.02 sekund
